In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import time
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn import metrics
from datetime import datetime
from IPython.display import HTML
from matplotlib import animation

# Ignore some warnings produced by basemap
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

# try to import Basemap
try:
    from mpl_toolkits.basemap import Basemap
    has_basemap = True
except ImportError:
    has_basemap = False

# Set the map resolution
# Notice: set resolution to full will make this program time-consuming
map_resolution = "i" # l: low, i: intermediate, h: high, f: full
    
plt.rcParams['figure.figsize'] = (20,5) # fiqure size

# Dataset
PM2.5 Dataset，every data has the following attributes: 
    - PM25: PM2.5 濃度
    - PM10: PM10 濃度
    - PM1 : PM1 濃度
    - Humidity: 溼度
    - Temperature: 溫度
    - DateTime: 時間
    - device_id: 感測器 id
    - record_id: 資料記錄 id
    - lon, lat: 感測器座標

In [2]:
with open('PM2.5.csv', mode='r', encoding='utf-8') as file:
    pm25_data = pd.read_csv(file, parse_dates=['DateTime'])

pm25_data.head()

,record_id,device_id,DateTime,PM25,PM10,PM1,Temperature,Humidity,lat,lon
0,12773,28C2DDDD415C,2017-03-01 08:30:00,43,0,0,19.61,96,23.741,120.755
1,12774,28C2DDDD415C,2017-03-01 09:00:00,41,0,0,19.81,94,23.741,120.755
2,12775,28C2DDDD415C,2017-03-01 09:30:00,41,0,0,19.73,93,23.741,120.755
3,12776,28C2DDDD415C,2017-03-01 10:00:00,47,0,0,19.17,92,23.741,120.755
4,12777,28C2DDDD415C,2017-03-01 10:30:00,50,0,0,19.90,91,23.741,120.755
